<font color='yellow' size=6></b>Parsing Member Transactions</b></font><br>
This notebook contains code snippets to assist in getting started parsing Member Transactions

In [1]:
import re 
import pandas as pd

In [2]:
card_tx = pd.read_csv('data/sample_card_comments.csv')

In [4]:
card_tx['Comments'].values[:10]

array(['2024-02-11 $9.82 || industrial motor & controls - west chester, ohio, united states #1417 ||% MCC 3348',
       '2024-01-25 $258.75 || mayfield & associates - solana beach, california, united states #6649 ||% MCC 4709',
       '2024-01-24 $181.51 || burdett metalsmithing & design - broomfield, colorado, united states #1004 ||% MCC 2437',
       '2024-02-16 $52.26 || the catalyst agency llc - odessa, florida, united states #1184 ||% MCC 1949',
       '2024-02-23 $144.80 || wisconsin turf equipment corp - new berlin, wisconsin, united states #3438 ||% MCC 5493',
       '2024-03-04 $227.94 || service electrical contractors, inc. - flushing, new york, united states #9451 ||% MCC 3643',
       '2024-02-08 $161.62 || wood technology center - nan #8670 ||% MCC 1144',
       '2024-02-22 $107.51 || ite @ umsl (innovative technology enterprises) - st. louis, missouri, united states #5345 ||% MCC 4429',
       '2024-03-16 $170.74 || ik solutions inc - newark, delaware, united states #4531

In [21]:
# ach_tx['Comments'].values[:10]

<font color='yellow' size=5>Extract Card Merchant</font>

In [6]:
def get_merchant_name(row):
    """
    Extract merchant's name from debit and credit card comments string.
    """
    merchant_name = re.search('\|\s.*\s-', row['Comments']).group(0)
    # Clean merchant name
    merchant_name = merchant_name.replace('| ', '') # remove pipes
    merchant_name = merchant_name.replace(' -', '') # remove dashes
    return merchant_name.strip() # remove whitespace

In [7]:
card_merchants = card_tx.apply(get_merchant_name, axis=1)

In [8]:
card_merchants.values[:25]

array(['industrial motor & controls', 'mayfield & associates',
       'burdett metalsmithing & design', 'the catalyst agency llc',
       'wisconsin turf equipment corp',
       'service electrical contractors, inc.', 'wood technology center',
       'ite @ umsl (innovative technology enterprises)',
       'ik solutions inc', 'tre native', 'chisholm exploration inc',
       'strategic business advisers', 'cosmetic procedures center',
       'summit childrens program inc', 'doris italian market & bakery',
       'railworks track systems inc',
       'cascade fresh cleaning services, llc', 'marsol technologies inc',
       'bryn athyn cathedral', 'k & d krueger farms & sons',
       'riverstone advisors, llc', 'champion graphics',
       'hotel st augustine', 'mona fashion', 'western flyer delivery'],
      dtype=object)

<font color='yellow' size=5>Extract Merchant Category Code</font>

In [18]:
def get_merchant_category_code(row):
    """
    Extract merchant's name from debit and credit card comments string.
    """
    mcc_code = re.search('\|%\sMCC\s\d{4}', row['Comments']).group(0)
    # Clean merchant name
    mcc_code = mcc_code.replace('|% MCC ', '') # remove pipes
    return int(mcc_code.strip()) # remove whitespace + convert to integer

In [19]:
mcc_codes = card_tx.apply(get_merchant_category_code, axis=1)

In [20]:
mcc_codes.values[:25]

array([3348, 4709, 2437, 1949, 5493, 3643, 1144, 4429, 2917, 5317, 2326,
       5529, 1067, 2210, 1184, 5119, 4984, 5082, 4827, 1758, 4015, 1816,
       1220, 1202, 1225], dtype=int64)

<font color='yellow' size=5>Extract Merchant Location</font>

In [46]:
def get_transaction_location(row):
    """
    Extract merchant location into an array of city, state, country.
    """
    tx_location = re.search('-\s.*,\s.*,\sunited\sstates\s#\d{4}', row['Comments']).group(0) # NOTE: limited to transactions in United States
    tx_location = tx_location.replace('- ', '') # remove leading dash via string replacement
    tx_location = re.sub('\s#\d{4}', '', tx_location) # remove last 4 of card via RegEx replacement
    return {key: value for key, value in zip(['city', 'state', 'country'], tx_location.split(', '))}

In [47]:
pd.DataFrame(card_tx.head().apply(get_transaction_location, axis=1).tolist()) # Parse first 5 records

,city,state,country
0,west chester,ohio,united states
1,solana beach,california,united states
2,broomfield,colorado,united states
3,odessa,florida,united states
4,new berlin,wisconsin,united states
